# Python для анализа данных

## Веб-скрэйпинг: скачивание файлов

*На основе блокнота Татьяны Рогович, НИУ ВШЭ*
*Автор: Ян Пиле, НИУ ВШЭ*

## Скачивание файлов

Кстати, еще одно применение скрэйпинга, о котором мы пока не поговорили - это автоматизированное скачивание файлов. Например, вы хотите скачать научные статьи по определенному ключевому слову или прайс-листы поставщика, которые он загружает на сайт в эскеле.

Давайте посмотрим, как скачивать файлы на примере pdf и заодно попробуем походить по ссылкам. Кстати, этот процесс еще часто называется spidering или crawling, потому что ваш скрипт как паучок ползет по ссылкам (отсюда и название первых поисковых роботов - spider).

Давайте попробуем сохранить англоязычные summary дисертаций, защищенных в 2019 году

Мы уже отредактировали фильтры и ссылка их запомнила. Позже сегодня посмотрим как можно самим заполнять такие поля с помощью Selenium.

https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes

In [16]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver as wb
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
driver = wb.Firefox()

driver.implicitly_wait(2)

In [17]:
link = 'https://www.hse.ru/sci/diss/?fulltext=yes'
driver.get(link)

In [37]:
soup = BeautifulSoup(driver.page_source)
# soup.find_all('a')

In [38]:
# soup.find_all('a', {'class':'link'})

Давайте для начала поэкспериментируем с первым кандидатом.

In [20]:
print(len(soup.find_all('a')))
soup.find_all('a', {'class':'link'})[:10]

190


[<a class="link">Коллаборационизм в Крыму в период нацистской оккупации (1941-1944 гг.)</a>,
 <a class="link" href="https://www.hse.ru/org/persons/25917795" target="_blank">Махалова Ирина Андреевна</a>,
 <a class="link" href="https://www.hse.ru/org/persons/22511135" target="_blank">Будницкий Олег Витальевич</a>,
 <a class="link">Вторая мировая война</a>,
 <a class="link">Крым</a>,
 <a class="link">коллаборационизм</a>,
 <a class="link">оккупация</a>,
 <a class="link">советская юстиция</a>,
 <a class="link">сталинизм</a>,
 <a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/xf/139/178/1583/диссертация_финал_Махалова.pdf">Диссертация</a>]

Видим, что ссылок очень много, а нам нужны только те, которые ведут на summary. Можно поискать их по тексту ссылки.

In [24]:
for link in soup.find_all('a', text='Summary'):
    print(link)

<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/09/24/1583178139/Summary_финал.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/08/18/1586370776/Казеев_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/09/10/1579195206/Неклюдов_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/10/02/1579198197/Резюме на английском_итог.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/07/13/1594067576/Соколова_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/10/01/1579193339/Summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/09/15/1579192441/Тюрин_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/08/21/1589222101/Гиринский_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" hre

И соберем ссылки в список.

In [31]:
links = []
for link in soup.find_all('a', text='Summary'):
    print(link)
    links.append(link.get('href'))
    
print(links)

<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/09/24/1583178139/Summary_финал.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/08/18/1586370776/Казеев_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/09/10/1579195206/Неклюдов_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/10/02/1579198197/Резюме на английском_итог.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/07/13/1594067576/Соколова_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/10/01/1579193339/Summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/09/15/1579192441/Тюрин_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2020/08/21/1589222101/Гиринский_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" hre

Отлично. Теперь нужно придумать, откуда взять название для каждого файла. Пусть это будут фамилии авторов, давайте доберемся до них. Такую задачу мы пока не решали: будем искать тэг по тексту, а потом искать его родителя (потому что ни ячейку таблицы с именем автора, ни саму таблицу не получится уникально отсечь).

In [26]:
for author in soup.find_all('td', text='Соискатель:'):
    print(author.parent())

[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href="https://www.hse.ru/org/persons/25917795" target="_blank">Махалова Ирина Андреевна</a></td>, <a class="link" href="https://www.hse.ru/org/persons/25917795" target="_blank">Махалова Ирина Андреевна</a>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href="https://www.hse.ru/org/persons/195203703" target="_blank">Казеев Никита Александрович</a></td>, <a class="link" href="https://www.hse.ru/org/persons/195203703" target="_blank">Казеев Никита Александрович</a>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href="https://www.hse.ru/org/persons/191263008" target="_blank">Неклюдов Кирилл Олегович</a></td>, <a class="link" href="https://www.hse.ru/org/persons/191263008" target="_blank">Неклюдов Кирилл Олегович</a>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href=

Достанем фамилии.

In [27]:
authors = []
for author in soup.find_all('td', text='Соискатель:'):
    print(author.parent()[1].get_text().split()[0])
    authors.append(author.parent()[1].get_text().split()[0])
    
print(authors)

Махалова
Казеев
Неклюдов
Драгой
Соколова
Коган
Тюрин
Гиринский
Дорофеева
Антипкина
['Махалова', 'Казеев', 'Неклюдов', 'Драгой', 'Соколова', 'Коган', 'Тюрин', 'Гиринский', 'Дорофеева', 'Антипкина']


Проверим, что списки действительно одинаковой длины.

In [29]:
len(links) == len(authors)

True

In [32]:
print(links)

['http://www.hse.ru/data/2020/09/24/1583178139/Summary_финал.pdf', 'http://www.hse.ru/data/2020/08/18/1586370776/Казеев_summary.pdf', 'http://www.hse.ru/data/2020/09/10/1579195206/Неклюдов_summary.pdf', 'http://www.hse.ru/data/2020/10/02/1579198197/Резюме на английском_итог.pdf', 'http://www.hse.ru/data/2020/07/13/1594067576/Соколова_summary.pdf', 'http://www.hse.ru/data/2020/10/01/1579193339/Summary.pdf', 'http://www.hse.ru/data/2020/09/15/1579192441/Тюрин_summary.pdf', 'http://www.hse.ru/data/2020/08/21/1589222101/Гиринский_summary.pdf', 'http://www.hse.ru/data/2020/09/14/1577837706/Summary.pdf', 'http://www.hse.ru/data/2020/09/11/1560161080/Summary.pdf']


Теперь попробуем сохранить файл. У нас все файлы pdf, будем переименовывать их фамилиями автора. Кстати, если файлы разного формата, то расширение можно узнать через атрибут headers

In [33]:
requests.get(links[0]).headers['content-type']

'application/pdf'

In [34]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True) 

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf': 

    # wb - запись байтовой информации
    fh = open('test.pdf', 'wb') 
    
    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content) 
    fh.close()

Давайте теперь еще добавим имя файла по фамилии.

In [35]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True) 

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf': 

    # wb - запись байтовой информации
    fh = open(f'{authors[0]}.pdf', 'wb') 
    
    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content) 
    fh.close()

Давайте сохраним обработку файла в функцию и соберем уже все в цикл.

In [36]:
def get_pdf(idx):
    summary = requests.get(links[idx], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
    if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
        fh = open(f'{authors[idx]}.pdf', 'wb') # wb - запись байтовой информации
        fh.write(summary.content) # считываем туда "содержание" файла по ссылке
        fh.close()

In [41]:
link = 'https://www.hse.ru/sci/diss/?fulltext=yes'
driver.get(link)

soup = BeautifulSoup(driver.page_source)

links = []
for link in soup.find_all('a', text='Summary'):
    links.append(link.get('href'))

authors = []
for author in soup.find_all('td', text='Соискатель:'):
    authors.append(author.parent()[1].get_text().split()[0])

In [44]:
links


['http://www.hse.ru/data/2020/09/24/1583178139/Summary_финал.pdf',
 'http://www.hse.ru/data/2020/08/18/1586370776/Казеев_summary.pdf',
 'http://www.hse.ru/data/2020/09/10/1579195206/Неклюдов_summary.pdf',
 'http://www.hse.ru/data/2020/10/02/1579198197/Резюме на английском_итог.pdf',
 'http://www.hse.ru/data/2020/07/13/1594067576/Соколова_summary.pdf',
 'http://www.hse.ru/data/2020/10/01/1579193339/Summary.pdf',
 'http://www.hse.ru/data/2020/09/15/1579192441/Тюрин_summary.pdf',
 'http://www.hse.ru/data/2020/08/21/1589222101/Гиринский_summary.pdf',
 'http://www.hse.ru/data/2020/09/14/1577837706/Summary.pdf',
 'http://www.hse.ru/data/2020/09/11/1560161080/Summary.pdf']

In [45]:
for idx in range(len(authors)):
    get_pdf(idx)

Готово!